Retrieval Augmented Generation meta-llama/Llama-2-7b-chat-hf with custom data

[HuggingFace Blog](https://huggingface.co/blog/llama2#how-to-prompt-llama-2)

In [1]:
!pip install llama-index transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.9/749.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()

In [ ]:
# from llama_index.node_parser import SimpleNodeParser

# parser = SimpleNodeParser.from_defaults()

# nodes = parser.get_nodes_from_documents(documents)

In [ ]:
# huggingface api token for downloading llama2
# hf_token = "hf_xxx"

In [3]:
from huggingface_hub import notebook_login

notebook_login()


In [4]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
    context_window=3900,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)


In [7]:
from llama_index.response.notebook_utils import display_response

query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("What are the steps for installing ELO server?")

display_response(response)

**`Final Response:`** Based on the provided context information, the steps for installing ELO server are as follows:

1. Create password lists and checklists to maintain overview and serve as proof of performance.
2. Set up accounts, including a database service account and an ELO Server Engine/Apache Tomcat service account.
3. Install Microsoft SQL Server or PostgreSQL.
4. Run ELO Server and execute it on the desired server computer.
5. Quick start: Perform the following steps for a quick ELO server installation:

a. Create a folder for the ELO installation.

b. Download ELO Server Setup and execute it in the ELO installation folder.

c. Set up accounts: You should set up the following accounts in advance:

- Database service account

- ELO Server Engine/Apache Tomcat service account

6. Troubleshooting: Check whether the ELO Web Client has loaded after the ELO Web Add-ons program is ready for operation. Use the browser console to verify whether the ELO Web Client is using the correct protocol and port. If you have configured HTTPS and are using self-signed certificates, your browser may classify the